In [ ]:
import pandas as pd

### 处理原始的10个种子下每个hadm_id以及相应的正误label

In [ ]:
adw_all = pd.read_excel(
    '/media/luzhenyang/project/agent_graph_diag/statistics/statistical_result_of_diagnosis.xlsx',
    sheet_name='Accuracy_easy_to_call_tool(bp)'
)
adw_all.columns

In [ ]:
import pandas as pd
import os

# 所有种子ID列表
seed_list = [1, 4, 7, 9, 10, 20, 23, 42, 71, 96]

# 建立一个总表来记录所有hadm_id和其seed
all_ids = []

for seed in seed_list:
    seed_path = f'/media/luzhenyang/project/agent_graph_diag/subset_ids_{seed}.csv'
    seed_df = pd.read_csv(seed_path)
    seed_df['seed'] = seed
    all_ids.append(seed_df)

id_all_df = pd.concat(all_ids, ignore_index=True)


In [ ]:
# 方法列表
methods = ['own', 'strict', 'lenient', 'FI', 'CoT']

# 初始化结果表
result_all = []

for seed in seed_list:
    for method in methods:
        file_path = f'/media/luzhenyang/project/agent_graph_diag/results/agent_graph_subset_{seed}_{method}.csv'
        
        if not os.path.exists(file_path):
            continue
        
        df = pd.read_csv(file_path)
        
        # 处理"对"和"错"：建议你确认下列的字段名
        if 'result' in df.columns:
            df['correct'] = df['result'].apply(lambda x: 1 if x == '对' else 0)
        elif '诊断正确' in df.columns:
            df['correct'] = df['诊断正确'].apply(lambda x: 1 if x == '对' else 0)
        else:
            raise ValueError(f"请检查{file_path}的列名，找不到结果字段")

        df['seed'] = seed
        df['method'] = method
        result_all.append(df[['hadm_id', 'seed', 'method', 'correct']])


In [ ]:
import pandas as pd

# 加载主表（诊断结果，只有Index，无hadm_id）
adw_all = pd.read_excel(
    '/media/luzhenyang/project/agent_graph_diag/statistics/statistical_result_of_diagnosis.xlsx',
    sheet_name='Accuracy_easy_to_call_tool(bp)'
)

# 所有种子编号和方法
seeds = [1, 4, 7, 9, 10, 20, 23, 42, 71, 96]
methods = ['own', 'strict', 'lenient', 'FI', 'CoT']

# 存储所有数据
records = []

# 遍历种子和方法
for seed in seeds:
    # 加载 hadm_id
    seed_id_path = f'/media/luzhenyang/project/agent_graph_diag/subset_ids_{seed}.csv'
    seed_df = pd.read_csv(seed_id_path)
    hadm_ids = seed_df['hadm_id'].tolist()

    for method in methods:
        col_name = f"{seed}_{method}"
        if col_name not in adw_all.columns:
            print(f"⚠️ 缺失列：{col_name}")
            continue
        
        # 获取当前列的结果（对/错/空）并转换为 0/1
        results = adw_all[col_name].fillna("").astype(str).str.strip()
        results_binary = results.apply(lambda x: 1 if x == '对' else 0)
        
        # 创建记录
        for i in range(len(hadm_ids)):
            records.append({
                'hadm_id': hadm_ids[i],
                'seed': seed,
                'method': method,
                'result': results_binary.iloc[i]
            })

# 构建长格式数据表
df_long = pd.DataFrame(records)

# 检查
print(df_long.head())
print(df_long['result'].value_counts())


In [ ]:
cli_longformer_pre = pd.read_csv('/media/luzhenyang/project/agent_graph_diag/lm_classification/training_logs_longformer_random_seeds/predictions_seed_1.csv')
print(cli_longformer_pre.columns)
print(cli_longformer_pre.shape)
print(cli_longformer_pre.head())

In [ ]:
import pandas as pd
import os
import re
from tqdm import tqdm

# 原主实验数据
# df_all = pd.read_csv(...)  # 请确保你事先加载了主表

# 临时列表收集所有 longformer 数据
longformer_list = []

# 所有种子
seeds = [1, 4, 7, 9, 10, 20, 23, 42, 71, 96]

# 预测文件夹路径
# base_path = '/media/luzhenyang/project/agent_graph_diag/lm_classification/training_logs_normal_longformer_rs'
base_path = '/media/luzhenyang/project/agent_graph_diag/lm_classification/training_logs_longformer_random_seeds'

for seed in tqdm(seeds):
    file_path = os.path.join(base_path, f'predictions_seed_{seed}.csv')
    if not os.path.exists(file_path):
        print(f"❗ 文件不存在: {file_path}")
        continue

    df = pd.read_csv(file_path)

    # 处理 hadm_id：从 tensor(12345678) 提取数字
    df['hadm_id'] = df['hadm_id'].astype(str).str.extract(r'(\d+)').astype(int)

    # 添加缺失列
    df['seed'] = seed
    df['method'] = 'clinical_longformer'
    df['result'] = (df['label'] == df['pred']).astype(int)

    # 保留所需列
    df = df[['hadm_id', 'seed', 'method', 'result']]

    # 去重：避免同一个 seed 下同一个 hadm_id 多次出现
    df = df.drop_duplicates(subset=['hadm_id', 'seed'])

    longformer_list.append(df)

# 合并所有 longformer 数据
df_longformer_all = pd.concat(longformer_list, ignore_index=True)

# 合并进主 df_all
df_all = pd.concat([df_long, df_longformer_all], ignore_index=True)

# 可选检查
print(df_longformer_all.shape)
print(df_longformer_all.duplicated(subset=['hadm_id', 'seed']).sum())

df_all  # 展示主表最终内容


In [ ]:
import pandas as pd
import os
import re
from tqdm import tqdm

# 原主实验数据
# df_all = pd.read_csv(...)  # 请确保你事先加载了主表

# 临时列表收集所有 longformer 数据
longformer_list = []

# 所有种子
seeds = [1, 4, 7, 9, 10, 20, 23, 42, 71, 96]

# 预测文件夹路径
base_path = '/media/luzhenyang/project/agent_graph_diag/lm_classification/training_logs_normal_longformer_rs'
# base_path = '/media/luzhenyang/project/agent_graph_diag/lm_classification/training_logs_longformer_random_seeds'

for seed in tqdm(seeds):
    file_path = os.path.join(base_path, f'predictions_seed_{seed}.csv')
    if not os.path.exists(file_path):
        print(f"❗ 文件不存在: {file_path}")
        continue

    df = pd.read_csv(file_path)

    # 处理 hadm_id：从 tensor(12345678) 提取数字
    df['hadm_id'] = df['hadm_id'].astype(str).str.extract(r'(\d+)').astype(int)

    # 添加缺失列
    df['seed'] = seed
    df['method'] = 'longformer'
    df['result'] = (df['label'] == df['pred']).astype(int)

    # 保留所需列
    df = df[['hadm_id', 'seed', 'method', 'result']]

    # 去重：避免同一个 seed 下同一个 hadm_id 多次出现
    df = df.drop_duplicates(subset=['hadm_id', 'seed'])

    longformer_list.append(df)

# 合并所有 longformer 数据
df_longformer_all = pd.concat(longformer_list, ignore_index=True)

# 合并进主 df_all
df_all = pd.concat([df_all, df_longformer_all], ignore_index=True)

# 可选检查
print(df_longformer_all.shape)
print(df_longformer_all.duplicated(subset=['hadm_id', 'seed']).sum())

df_all  # 展示主表最终内容


In [ ]:
df_all['method'].value_counts()

In [ ]:
df_all.to_csv('/media/luzhenyang/project/agent_graph_diag/GLMM/df_all.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import numpy as np # Keep numpy for potential future use

# --- STEP 1: LOAD YOUR REAL DATA (替换掉整个模拟部分) ---
# 假设你所有的真实实验结果都保存在这个文件里
real_results_path = "/media/luzhenyang/project/agent_graph_diag/GLMM/df_all.csv" 
df_all = pd.read_csv(real_results_path)

# --- 检查数据，确保一切正确 ---
# 你可以取消下面这行的注释来验证
# unique_patients = df_all['hadm_id'].nunique()
# print(f"Successfully loaded data. Found {unique_patients} unique patients.")
# 如果这里打印出 659，那就对了！

# --- 从这里开始，后面的代码基本不变 ---

# 设定方法的比较基准 (reference group)
all_methods = ['own', 'lenient', 'strict', 'FI', 'CoT', 'clinical_longformer', 'longformer']
# all_methods = ['ADW-own', 'ADW-lenient', 'ADW-strict', 'Vanilla Prompting', 'CoT', 'Clinical Longformer', 'Longformer']
df_all['method'] = pd.Categorical(df_all['method'], categories=all_methods, ordered=True)

# 拟合 GEE 模型
print("--- Fitting GEE model on REAL experimental data ---")
model_gee = smf.gee(
    "result ~ method", 
    data=df_all, 
    groups=df_all["hadm_id"], # GEE会在这里找到659个groups
    family=sm.families.Binomial(),
    cov_struct=sm.cov_struct.Exchangeable()
)
result_gee = model_gee.fit()

print("--- GEE model fitting successful! ---")
# 这一次，这里的输出应该会显示 "No. clusters: 659"
print(result_gee.summary())


# ----------------------
# ✅ 3. 提取并整理结果
# ----------------------
params = result_gee.params
conf_int = result_gee.conf_int()
p_values = result_gee.pvalues

# 计算 Odds Ratios
odds_ratios = np.exp(params)
or_low_ci = np.exp(conf_int[0])
or_high_ci = np.exp(conf_int[1])

# 创建一个清晰的结果表格
results_table = pd.DataFrame({
    'Method': params.index,
    'Log(OR)': params.values,
    'P-value': p_values.values,
    'Odds Ratio (OR)': odds_ratios.values,
    '95% CI Low': or_low_ci.values,
    '95% CI High': or_high_ci.values
}).reset_index(drop=True)

# 把基准（Intercept）的名字改成更清晰的 'own (Reference)'
results_table.loc[results_table['Method'] == 'Intercept', 'Method'] = f"{all_methods[0]} (Reference)"
results_table.loc[results_table['Method'] == f"{all_methods[0]} (Reference)", 'Odds Ratio (OR)'] = 1.0
results_table.loc[results_table['Method'] == f"{all_methods[0]} (Reference)", ['95% CI Low', '95% CI High']] = 1.0


# 清理其他方法的名称
results_table['Method'] = results_table['Method'].str.replace('method[T.', '', regex=False).str.replace(']', '', regex=False)

# ✅ 3.1. 保存统计结果到 CSV 文件
# ----------------------
results_filename = "/media/luzhenyang/project/agent_graph_diag/GLMM/diagnostic_methods_gee_results.csv"
# results_table.to_csv(results_filename, index=False)
print(f"\n--- Statistical results saved to {results_filename} ---")

print("\n--- Final Odds Ratios Table for Manuscript ---")
print(results_table)


# ----------------------
# ✅ 4. 可视化结果 (森林图 Forest Plot)
# ----------------------
# 准备绘图用的数据框，确保顺序正确
plot_df = results_table.copy()
plot_df['Method'] = plot_df['Method'].replace({'FI': 'Vanilla Prompting'})
plot_df['Method'] = plot_df['Method'].replace({'own (Reference)': 'ADW-own'})
plot_df['Method'] = plot_df['Method'].replace({'lenient': 'ADW-lenient'})
plot_df['Method'] = plot_df['Method'].replace({'strict': 'ADW-strict'})
plot_df['Method'] = plot_df['Method'].replace({'CoT': 'Chain-of-Thought'})
plot_df['Method'] = plot_df['Method'].replace({'clinical_longformer': 'Clinical Longformer'})
plot_df['Method'] = plot_df['Method'].replace({'longformer': 'Longformer'})


plot_df['Method'] = pd.Categorical(plot_df['Method'], categories=plot_df['Method'].tolist(), ordered=True)
plot_df = plot_df.sort_values(by='Method', ascending=False) # 倒序，这样在图上是正序

plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(10, 6))

# 绘制点和误差线
ax.errorbar(
    x=plot_df['Odds Ratio (OR)'],
    y=plot_df['Method'],
    xerr=[plot_df['Odds Ratio (OR)'] - plot_df['95% CI Low'], plot_df['95% CI High'] - plot_df['Odds Ratio (OR)']],
    fmt='o', # 'o' for point marker
    color='black',
    ecolor='black',
    capsize=5,
    elinewidth=1.5,
    markersize=7,
    markerfacecolor='royalblue',
    markeredgecolor='black'
)

# 绘制无效线
ax.axvline(x=1.0, linestyle='--', color='red', linewidth=1.2)

# 设置对数刻度
ax.set_xscale('log')

# 美化图表
# ax.set_title('Comparison of Diagnostic Methods vs. Reference (ADW-own)', fontsize=16, fontweight='bold')
ax.set_xlabel('Odds Ratio (OR) and 95% Confidence Interval (Log Scale)', fontsize=12)
ax.set_ylabel('Method', fontsize=12)
# 根据你的方法数量调整x轴范围和刻度
ax.set_xticks([0.1, 0.5, 1.0, 2.0, 5.0])
ax.get_xaxis().set_major_formatter(plt.ScalarFormatter())

# 在图的右侧添加OR和CI的文字标注，更专业
for i, row in plot_df.iterrows():
    ax.text(
        ax.get_xlim()[1] * 1.1, # 将文本放在图右侧
        row['Method'],
        f"{row['Odds Ratio (OR)']:.2f} ({row['95% CI Low']:.2f} - {row['95% CI High']:.2f})",
        va='center',
        ha='left'
    )

plt.tight_layout()
# ✅ 4.1. 保存高质量图片
# ----------------------
# dpi=300 是大多数期刊要求的最低分辨率
figure_filename = "/media/luzhenyang/project/agent_graph_diag/GLMM/diagnostic_methods_forest_plot_v2.png" # TIFF 或 PDF 格式更适合出版
plt.savefig(figure_filename, dpi=300, bbox_inches='tight') # bbox_inches='tight' 避免标签被截断
print(f"--- Forest plot saved to {figure_filename} ---")
plt.show()

In [ ]:
gee_stats = pd.read_csv('/media/luzhenyang/project/agent_graph_diag/GLMM/diagnostic_methods_gee_results.csv')
print(gee_stats)

In [ ]:
import numpy as np
import statsmodels.stats.power as smp
import pandas as pd

# --- Helper Functions ---
def cohen_h(p1, p2):
    """Calculates Cohen's h for effect size between two proportions."""
    return 2 * (np.arcsin(np.sqrt(p1)) - np.arcsin(np.sqrt(p2)))

def log_odds_to_prob(log_odds):
    """Converts log odds to probability."""
    return np.exp(log_odds) / (1 + np.exp(log_odds))

# --- Parameters from your FINAL GEE results ---
alpha = 0.05

# Log_odds for the reference group ('own') is the Intercept from the GEE output
log_odds_own = 2.5483
p_own = log_odds_to_prob(log_odds_own)

# Calculate log_odds for all comparison groups
all_methods_log_odds = {
    'strict': log_odds_own - 1.5027,
    'lenient': log_odds_own - 0.1886,
    'FI': log_odds_own - 0.7335,
    'CoT': log_odds_own - 0.2654,
    'clinical_longformer': log_odds_own - 3.2909,
    'longformer': log_odds_own - 3.3024
}

# Convert log_odds to probabilities to display in the table
all_methods_prob = {name: log_odds_to_prob(lo) for name, lo in all_methods_log_odds.items()}

# --- Key Sample Sizes ---
n_gee_correct = 659  # The correct number of unique patients (clusters)
n_naive_flawed = 10  # The flawed number of sample runs for comparison

# --- Analysis 1: Power of the Correct GEE Model ---
print(f"Calculating power for GEE model with n = {n_gee_correct} unique patients...")
power_gee_final = {}
for name, p_method in all_methods_prob.items():
    effect_size_h = cohen_h(p_own, p_method)
    power = smp.NormalIndPower().power(
        effect_size=abs(effect_size_h), 
        nobs1=n_gee_correct, 
        alpha=alpha,
        ratio=1.0, 
        alternative='two-sided'
    )
    power_gee_final[name] = power

# --- Analysis 2: Power of the Naive Test ---
print(f"Calculating power for naive test with n = {n_naive_flawed} sample runs...")
power_naive_final = {}
for name, p_method in all_methods_prob.items():
    effect_size_h = cohen_h(p_own, p_method)
    power = smp.TTestIndPower().power(
        effect_size=abs(effect_size_h),
        nobs1=n_naive_flawed,
        alpha=alpha,
        ratio=1.0, 
        alternative='two-sided'
    )
    power_naive_final[name] = power

# --- Create the Final Comparison Table ---
df_power_final = pd.DataFrame({
    'Comparison': [f"'own' vs. '{name}'" for name in all_methods_prob.keys()],
    'Observed Accuracy (own)': [f"{p_own:.1%}" for _ in all_methods_prob],
    'Observed Accuracy (Comparison)': [f"{p:.1%}" for p in all_methods_prob.values()],
    f'Power of GEE Model (n={n_gee_correct})': [f"{power_gee_final[name]:.1%}" for name in all_methods_prob.keys()],
    f'Power of Naive Test (n={n_naive_flawed})': [f"{power_naive_final[name]:.1%}" for name in all_methods_prob.keys()]
})

# --- Sort table by GEE power for clear presentation ---
gee_power_numeric = [power_gee_final[name] for name in all_methods_prob.keys()]
df_power_final['gee_power_numeric'] = gee_power_numeric
df_power_final = df_power_final.sort_values(by='gee_power_numeric', ascending=False).drop(columns=['gee_power_numeric'])

# --- Display the final results ---
print("\n" + "="*50)
print("  Final Post-hoc Statistical Power Analysis Table  ")
print("="*50)
print(df_power_final.to_markdown(index=False, tablefmt="grid"))